In [55]:
#Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
#Read csv file
df = pd.read_csv('data_proj3.csv')
df.head()

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Clarify Item No,IBase Number,RMA Item Count,Product Family,Product ID,Theater,RMA Request Date,New Replacement - Delivered On Date,Replacement - Delivered On Date,RMA Create Date,...,Contract ID,As Maintained2 Ship Date - IB,As Built2 Ship Date,SR Customer ID,SR Customer Name,Serial Number,Item Status,Installed At Name Reference Only - Text,RMA Item Transaction Type,Item No
0,#,18427259,1.0,EX,81Y8237,APAC,4/18/2017,NaN,4/4/2017,3/20/2017,...,#,10/12/2012,10/12/2012,100034495,JUNIPER NETWORKS - INTERNAL - APAC SERVICE LOG...,233G04B,RECEIVED,IBM SINGAPORE PTE,RMA Defective Item,100.0
1,#,18427259,0.0,EX,EX4500-40F-BF-C,APAC,NaN,NaN,4/4/2017,3/20/2017,...,#,10/12/2012,10/12/2012,100034495,JUNIPER NETWORKS - INTERNAL - APAC SERVICE LOG...,GG0211059011,REPLACEMENT DELIVERED,IBM SINGAPORE PTE,RMA Replacement Item,110.0
2,#,2727562,1.0,EX,81Y8349,AMER,2/22/2018,NaN,2/28/2018,1/31/2018,...,#,7/18/2011,7/18/2011,101400565,DROP IN SOLUTIONS,231S01X,RECEIVED,IBM CORPORATION,RMA Defective Item,100.0
3,#,4769193,1.0,EX,95Y0510,EMEA,5/9/2017,NaN,NaN,4/12/2017,...,#,10/29/2013,10/29/2013,101183390,IBM (HUNGARY),EM0711427457,RECEIVED,IBM CORPORATION,RMA Defective Item,100.0
4,#,8928104,1.0,EX,95Y0510,EMEA,5/9/2017,NaN,NaN,4/27/2017,...,#,12/13/2012,12/13/2012,101183390,IBM (HUNGARY),EM0711427505,RECEIVED,IBM CORPORATION,RMA Defective Item,100.0


In [11]:
df.shape

(198768, 39)

In [6]:
df.columns

Index(['Clarify Item No', 'IBase Number', 'RMA Item Count', 'Product Family',
       'Product ID', 'Theater', 'RMA Request Date',
       'New Replacement - Delivered On Date',
       'Replacement - Delivered On Date', 'RMA Create Date',
       'RMA Create Quarter', 'RMA Create Month', 'Country', 'Service Level',
       'OTD Status', 'OTD Grouping', 'OTD Service Level', 'Header Status',
       'RMA Type', 'RMA Delivery Due Date', 'RMA ID', 'RMA Ship Site ID',
       'Replacement - RMA Ship Date', 'RMA Header Create Date',
       'RMA Request Date Month', 'Ship- to Party - IB - Text',
       'Ship- to Party - IB - Key', 'Original Product Ship Date',
       'Installed At Name Reference Only - Key', 'Contract ID',
       'As Maintained2 Ship Date - IB', 'As Built2 Ship Date',
       'SR Customer ID', 'SR Customer Name', 'Serial Number', 'Item Status',
       'Installed At Name Reference Only - Text', 'RMA Item Transaction Type',
       'Item No'],
      dtype='object')

In [16]:
#Recreate DF inplace with only columns of interest
df = df[['RMA Item Count', 'Product Family', 'Product ID', 'RMA Create Date', 'RMA Delivery Due Date', 'Country', 'Service Level', 'RMA Ship Site ID', 'Installed At Name Reference Only - Key', 'As Built2 Ship Date', 'Serial Number', 'RMA Item Transaction Type']]
df.head()

,RMA Item Count,Product Family,Product ID,RMA Create Date,RMA Delivery Due Date,Country,Service Level,RMA Ship Site ID,Installed At Name Reference Only - Key,As Built2 Ship Date,Serial Number,RMA Item Transaction Type
0,1.0,EX,81Y8237,3/20/2017,4/3/2017,Singapore,IWR-RTF,IBM SINGAPORE PTE,101276652,10/12/2012,233G04B,RMA Defective Item
1,0.0,EX,EX4500-40F-BF-C,3/20/2017,4/3/2017,Singapore,IWR-RTF,IBM SINGAPORE PTE,101276652,10/12/2012,GG0211059011,RMA Replacement Item
2,1.0,EX,81Y8349,1/31/2018,2/14/2018,USA,IWR-RTF,IBM CORPORATION,100521308,7/18/2011,231S01X,RMA Defective Item
3,1.0,EX,95Y0510,4/12/2017,4/26/2017,Spain,IWR-RTF,IBM CORPORATION,100521308,10/29/2013,EM0711427457,RMA Defective Item
4,1.0,EX,95Y0510,4/27/2017,5/11/2017,Spain,IWR-RTF,IBM CORPORATION,100521308,12/13/2012,EM0711427505,RMA Defective Item


In [44]:
#Filter for RMA Defective Item only.... this step might need to be removed
df_defect = df.loc[df['RMA Item Transaction Type'] == 'RMA Defective Item']

#ONLY ONE PRODUCT ID FOR NOW
df_defect = df.loc[df['Product ID'] == 'EX3300-48P']
df_defect.head(30)

,RMA Item Count,Product Family,Product ID,RMA Create Date,RMA Delivery Due Date,Country,Service Level,RMA Ship Site ID,Installed At Name Reference Only - Key,As Built2 Ship Date,Serial Number,RMA Item Transaction Type
183,1.0,EX,EX3300-48P,9/11/2017,NaN,India,NOT REPORTED,FLIPKART INDIA PVT,101023446,NaN,Not assigned,RMA Defective Item
184,0.0,EX,EX3300-48P,10/9/2017,10/10/2017,Australia,NEXT DAY,DEPARTMENT OF,101237462,NaN,Not assigned,RMA Replacement Item
185,0.0,EX,EX3300-48P,10/25/2017,NaN,Malaysia,DOA/DOA-REPL,ZF CHASSIS SYSTEMS,101420883,NaN,GB0217280024,RMA Replacement Item
186,1.0,EX,EX3300-48P,4/9/2018,NaN,India,NOT REPORTED,FLIPKART INDIA PVT,101023446,NaN,Not assigned,RMA Defective Item
187,0.0,EX,EX3300-48P,11/23/2017,NaN,India,NOT REPORTED,JUNIPER NETWORKS,101314213,NaN,Not assigned,RMA Replacement Item
188,0.0,EX,EX3300-48P,8/21/2017,NaN,USA,DOA/DOA-REPL,LINCOLN PARISH,101308999,NaN,GB0217190552,RMA Replacement Item
189,1.0,EX,EX3300-48P,10/22/2017,10/24/2017,USA,NEXT DAY,PUBLIC HEALTH,101332913,NaN,Not assigned,RMA Defective Item
190,0.0,EX,EX3300-48P,2/19/2018,2/20/2018,USA,IWR-ND SHIP,MIAMI PS,101277395,NaN,GB0217190029,RMA Replacement Item
191,0.0,EX,EX3300-48P,6/19/2018,6/20/2018,Brazil,IWR-ND SHIP,STEMAC GRUPOS,100066069,NaN,Not assigned,RMA Replacement Item
192,0.0,EX,EX3300-48P,6/19/2018,6/20/2018,Brazil,IWR-ND SHIP,STEMAC GRUPOS,100066069,NaN,Not assigned,RMA Replacement Item


In [43]:
# df_defect.groupby('Product ID').count().sort_values('RMA Item Count', ascending=False)
df_defect.isnull().sum()

RMA Item Count                              0
Product Family                              0
Product ID                                  0
RMA Create Date                             0
RMA Delivery Due Date                     120
Country                                     0
Service Level                               0
RMA Ship Site ID                            0
Installed At Name Reference Only - Key      0
As Built2 Ship Date                       115
Serial Number                               0
RMA Item Transaction Type                   0
dtype: int64

In [46]:
#taking only non NAN rows
df_defect = df_defect[pd.notnull(df['As Built2 Ship Date'])]
df_defect

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,RMA Item Count,Product Family,Product ID,RMA Create Date,RMA Delivery Due Date,Country,Service Level,RMA Ship Site ID,Installed At Name Reference Only - Key,As Built2 Ship Date,Serial Number,RMA Item Transaction Type
28603,0.0,EX,EX3300-48P,6/9/2017,6/9/2017,France,SAME DAY,AON FRANCE,101369144,6/3/2016,GB0216210834,Actual Defective Received
28941,0.0,EX,EX3300-48P,1/24/2017,1/25/2017,USA,IWR-ND SHIP,SECUREMATICS,100520459,3/15/2016,GB0215480415,RMA Replacement Item
42477,0.0,EX,EX3300-48P,9/26/2017,10/10/2017,Russian Fed.,RTF,MRC OF ICT,101157968,5/17/2017,GB0217190161,RMA Replacement Item
42672,1.0,EX,EX3300-48P,3/5/2018,3/6/2018,Australia,NEXT DAY,TELSTRA--DOD,100118799,10/5/2017,GB0217250339,RMA Defective Item
43309,1.0,EX,EX3300-48P,11/16/2017,NaN,USA,NOT REPORTED,UNIVERSITY OF,100942550,12/18/2015,GB0215381037,RMA Defective Item
43310,0.0,EX,EX3300-48P,5/23/2018,5/24/2018,Utd.Arab Emir.,IWR-ND SHIP,EMIRATES PALACE,100993926,12/18/2015,GB0215381037,RMA Replacement Item
43311,1.0,EX,EX3300-48P,10/24/2017,10/25/2017,USA,IWR-ND SHIP,CECIL COUNTY,101322152,11/29/2016,GB0216460648,RMA Defective Item
43312,0.0,EX,EX3300-48P,2/16/2017,2/17/2017,USA,IWR-ND SHIP,NATIONAL SPINE AND,101151012,6/4/2015,GB0214500893,RMA Replacement Item
43313,0.0,EX,EX3300-48P,9/25/2017,9/26/2017,USA,NEXT DAY,CONEJO VALLEY,100739362,11/5/2014,GB0214281398,RMA Replacement Item
43314,0.0,EX,EX3300-48P,10/1/2017,10/3/2017,Nigeria,IWR-ND SHIP,PAN ATLANTIC,100027821,11/5/2014,GB0214281398,RMA Replacement Item


In [47]:
#Drop RMA Item type to create 'df_x' with columns for all meta_data
df_x = df_defect.drop('RMA Item Transaction Type', 1)
df_x.head()


#TESTING SECTION... further reduce meta_data columns for testing ONE HOT ENCODING
df_x1 = df_x[['Product ID', 'Country', 'RMA Ship Site ID', 'As Built2 Ship Date', 'Serial Number']]

In [59]:
country = df_x1['Country'].values
site_id = df_x1['RMA Ship Site ID']

In [60]:
#label encoder
label_encoder = LabelEncoder()

label_encoder.fit(country)
encoded_country = label_encoder.transform(country)

label_encoder.fit(site_id)
encoded_site = label_encoder.transform(site_id)

In [61]:
one_hot_country = to_categorical(encoded_country)
one_hot_site = to_categorical(encoded_site)
one_hot_country

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [62]:
one_hot_site

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [72]:
one_hot_site[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [67]:
df_x1['Hot_Country'] = pd.Series(one_hot_country[0])
df_x1['Hot_Site'] = pd.Series(one_hot_site[0])
df_x1.head()

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Product ID,Country,RMA Ship Site ID,As Built2 Ship Date,Serial Number,Hot_Country,Hot_Site
28603,EX3300-48P,France,AON FRANCE,6/3/2016,GB0216210834,NaN,NaN
28941,EX3300-48P,USA,SECUREMATICS,3/15/2016,GB0215480415,NaN,NaN
42477,EX3300-48P,Russian Fed.,MRC OF ICT,5/17/2017,GB0217190161,NaN,NaN
42672,EX3300-48P,Australia,TELSTRA--DOD,10/5/2017,GB0217250339,NaN,NaN
43309,EX3300-48P,USA,UNIVERSITY OF,12/18/2015,GB0215381037,NaN,NaN


In [70]:
df_x1['Hot_Country'].values[0]

nan

In [48]:
y = df_defect['RMA Item Transaction Type'].values.reshape(-1,1)
X = df_x1
print(X.shape, y.shape)

(4530, 5) (4530, 1)


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [41]:
model = LinearRegression()
model.fit(X_train, y_train)
train_score = model.score(X_train, y_train)
testing_Score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

ValueError: could not convert string to float: 'GB0216360145'